In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## LABEL DATA

import pandas as pd
import numpy as np

labels = pd.read_csv('/content/drive/MyDrive/introToFADS/week8/studentLife_no-audio__no-wifi/studentLife/survey/PerceivedStressScale.csv')

# there are 39 post responses and 46 pre responses
labelsproc = np.empty(39, dtype=np.int64)


# takes THE PerceivedStressedScale.csv and a zeroed array, populates zeroed array
# with stress scores
def surveyIntoStressScore(X, y):
    
    arr = np.ones((39,9))
    colLabels = X.columns.values.tolist()
    
    # converts every survey answer into scores
    # populates arr with scores
    for i in range(0,39):
        for j in range(2,11):
            arr[i,j-2] = textToScore(X[colLabels[j]][46+i])
        
    sumArr = np.sum(arr, axis=1)
        
    return sumArr;  

def textToScore(str):
    if str == 'Never':
        return 0;
    elif str == 'Almost never':
        return 1;
    elif str == 'Sometime':
        return 2;
    elif str == 'Fairly often':
        return 3;
    elif str == 'Very often':
        return 4;
    

labelsproc.size
labelVector = surveyIntoStressScore(labels,labelsproc)
# print(labelVector)
# prints out label vector containing stress scores of 
# participants who replied to the post Stress




## GPS DATA

import pandas as pd
import numpy as np
import math

# 1->'01' 25->'25'
def doubleDigitizer(x):
    if x < 10:
        return '0' + str(x);
    else:
        return str(x);

# to identify missing
missing_vectors = np.array([])

    
# here are 60 users from gps_u00 to gps_u59
# populate df_collection with every gps data for every user
df_collection = {}
for i in range(0,60):
    ddi = doubleDigitizer(i)
    try:
        df_collection[ddi] = pd.read_csv('/content/drive/MyDrive/introToFADS/week8/studentLife_no-audio__no-wifi/studentLife/sensing/gps/gps_u' + ddi + '.csv')
    except IOError:
        print "no user # " + ddi + " for GPS"
        missing_vectors = np.append(missing_vectors,ddi)
        pass
        

        

def distBetweenTwoCoords(lat1,lon1,lat2,lon2):
    
    R = 6371 # Radius of Earth in km

    dLat = math.radians(float(lat2) - float(lat1))
    dLon = math.radians(float(lon2) - float(lon1))
    lat1 = math.radians(float(lat1))
    lat2 = math.radians(float(lat2))
    
    a = math.sin(dLat/2) * math.sin(dLat/2) + \
        math.cos(lat1) * math.cos(lat2) * math.sin(dLon/2) * math.sin(dLon/2)

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    d = R * c

    return d;

#assumes input is DataFrame, with labeled latitude and longitude columns

def totalDistCovered(X):
    
    sum = 0
    
    lats = X['latitude']
    lons = X['longitude']
    
    for i in range(0,(lats.size-1)):
        sum += distBetweenTwoCoords(lats.get_values()[i],lons.get_values()[i],lats.get_values()[i+1],lons.get_values()[i+1])
                
    return sum;
df_collection2={}
for i in range(0,60):
    ddi = doubleDigitizer(i)
    try:
        df_collection2[ddi] = pd.read_csv('/content/drive/MyDrive/introToFADS/week8/studentLife_no-audio__no-wifi/studentLife/sensing/activity/activity_u' + ddi + '.csv')

    except IOError:
      pass
      print "no user # " + ddi + " for activity"

def activityE(X):
  activity_value = 0
  activity_value = sum(X[' activity inference'].get_values())
  return activity_value

activityFeatureVector = np.zeros((60,1))
for i in range(0,60):
  ddi = doubleDigitizer(i)
  try:
    activityFeatureVector[i,0]=  activityE(df_collection2[ddi])
  except KeyError:
    activityFeatureVector[i,0] = np.NaN

gpsFeatureVector = np.zeros((60,1))

for i in range(0,60):
    ddi = doubleDigitizer(i)
    try:
        gpsFeatureVector[i,0] = totalDistCovered(df_collection[ddi])
    except KeyError:
        gpsFeatureVector[i,0] = np.NaN
    



## smsDATA

import pandas as pd
import numpy as np
import math

# 1->'01' 25->'25'
def doubleDigitizer(x):
    if x < 10:
        return '0' + str(x);
    else:
        return str(x);

# here are 60 users from gps_u00 to gps_u59
# populate df_collection with every gps data for every user
df_collection = {}
for i in range(0,60):
    ddi = doubleDigitizer(i)
    try:
        df_collection[ddi] = pd.read_csv('/content/drive/MyDrive/introToFADS/week8/studentLife_no-audio__no-wifi/studentLife/sms/sms_u' + ddi + '.csv')
    except IOError:
        pass
        print "no user # " + ddi + " for SMS"
        
from datetime import datetime

def timeStampToDatetime(ts):
    return datetime.utcfromtimestamp(float(ts))

# outputs datetime.datetime(2013, 3, 24, 4, 31, 22)
# that's for year, month, day, hour, minute, second

# int1 is first time, int2 is last time, returns days elapsed in between
def daysInBetween(int1, int2):
    secs = int2 - int1
    return secs/86400;


def dailySmsFreq(X):
    ts = X['timestamp']
    return (ts.size)/daysInBetween(ts[0], ts[ts.size-1]);


smsFeatureVector = np.zeros((60,1))

for i in range(0,60):
    ddi = doubleDigitizer(i)
    try:
        smsFeatureVector[i,0] = dailySmsFreq(df_collection[ddi])
    except KeyError:
        smsFeatureVector[i,0] = np.NaN
    
## CALLLOG DATA

import pandas as pd
import numpy as np
import math

def doubleDigitizer(x):
    if x < 10:
        return '0' + str(x);
    else:
        return str(x);

# here are 60 users from gps_u00 to gps_u59
# populate df_collection with every gps data for every user
df_collection = {}
for i in range(0,60):
    ddi = doubleDigitizer(i)
    try:
        df_collection[ddi] = pd.read_csv('/content/drive/MyDrive/introToFADS/week8/studentLife_no-audio__no-wifi/studentLife/call_log/call_log_u' + ddi + '.csv')
    except IOError:
        pass
        print "no user # " + ddi + " for CALL"
        
from datetime import datetime

def timeStampToDatetime(ts):
    return datetime.utcfromtimestamp(float(ts))

# outputs datetime.datetime(2013, 3, 24, 4, 31, 22)
# that's for year, month, day, hour, minute, second

# int1 is first time, int2 is last time, returns days elapsed in between
def daysInBetween(int1, int2):
    secs = int2 - int1
    return secs/86400;

def dailyCallFreq(X):
    ts = X['timestamp']
    return (ts.size)/daysInBetween(ts[0], ts[ts.size-1]);

# dailyCallFreq(df_collection['00'])

callFeatureVector = np.zeros((60,1))

for i in range(0,60):
    ddi = doubleDigitizer(i)
    try:
        callFeatureVector[i,0] = dailyCallFreq(df_collection[ddi])
    except KeyError:
        callFeatureVector[i,0] = np.NaN
    

a1 = gpsFeatureVector
a2 = activityFeatureVector
a3 = smsFeatureVector
a4 = callFeatureVector


no user # 06 for GPS
no user # 11 for GPS
no user # 21 for GPS
no user # 26 for GPS
no user # 28 for GPS
no user # 29 for GPS
no user # 37 for GPS
no user # 38 for GPS
no user # 40 for GPS
no user # 48 for GPS
no user # 55 for GPS
no user # 06 for activity
no user # 11 for activity
no user # 21 for activity
no user # 26 for activity
no user # 28 for activity
no user # 29 for activity
no user # 37 for activity
no user # 38 for activity
no user # 40 for activity
no user # 48 for activity
no user # 55 for activity
no user # 06 for SMS
no user # 11 for SMS
no user # 21 for SMS
no user # 26 for SMS
no user # 28 for SMS
no user # 29 for SMS
no user # 37 for SMS
no user # 38 for SMS
no user # 40 for SMS
no user # 48 for SMS
no user # 55 for SMS
no user # 06 for CALL
no user # 11 for CALL
no user # 21 for CALL
no user # 26 for CALL
no user # 28 for CALL
no user # 29 for CALL
no user # 37 for CALL
no user # 38 for CALL
no user # 40 for CALL
no user # 48 for CALL
no user # 55 for CALL


In [3]:
featureMatrix = np.hstack((a1,a2,a3,a4))

## KNN

from sklearn.metrics import accuracy_score
import operator
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3)


def ffill_cols(a, startfillval=0):
    mask = np.isnan(a)
    tmp = a[0].copy()
    a[0][mask[0]] = startfillval
    mask[0] = False
    idx = np.where(~mask,np.arange(mask.shape[0])[:,None],0)
    out = np.take_along_axis(a,np.maximum.accumulate(idx,axis=0),axis=0)
    a[0] = tmp
    return out

# removing NULLs
missing_vectors = np.array([6,11,21,26,28,29,37,38,40,48,55])

for i in range(0,missing_vectors.size):
    
    featureMatrix = np.delete(featureMatrix, int(missing_vectors[i])-i, 0)

    pass



cutoff = 20
print(featureMatrix.shape)
# shuffles matrix
np.random.shuffle(featureMatrix)

train = featureMatrix[0:cutoff]

test = featureMatrix[cutoff:39]

trainl = labelVector[0:cutoff]
testl = labelVector[cutoff:39]
knn.fit(train, trainl)
ypred = knn.predict(test)

# print(result)

print(accuracy_score(testl, ypred))

print(1-(float(abs(sum(map(operator.sub,testl,ypred))))/float(sum(testl))))

(49, 4)
0.2631578947368421
0.974285714286


In [4]:
from sklearn import metrics


cutoff = 20

train = featureMatrix[0:cutoff]
test = featureMatrix[cutoff:39]

trainl = labelVector[0:cutoff]
testl = labelVector[cutoff:39]


# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# fit the model with data
logreg.fit(train, trainl)

# predict the response values for the observations in X
y_pred = logreg.predict(test)
print(metrics.accuracy_score(testl, y_pred))

0.0


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
